In [1]:
import os
import pydicom
import matplotlib.pyplot as plt

plt.set_cmap('gray')
plt.rcParams['figure.figsize'] = (18, 12)

from tqdm import tqdm_notebook as tqdm
from glob import glob
import pandas as pd
import numpy as np

import subtle.utils.io as suio
import subtle.subtle_preprocess as supre

import ipywidgets as widgets
from ipywidgets import interact, interact_manual
from IPython.display import display, clear_output


def get_dicom_vol(dirpath_dicom):
    dcm_files = sorted([f for f in glob('{}/*.dcm'.format(dirpath_dicom))])
    return np.array([pydicom.dcmread(f).pixel_array for f in dcm_files])

def get_pp_path(inf_rec):
    cnum = inf_rec['Case']
    pp_path = '/'.join(inf_rec['Inference_Path'].split('/')[:-1]).replace('data', 'preprocess/data')
    pp_path = '{}/{}'.format(pp_path, cnum)
    file_ext = 'npy' if os.path.exists('{}.npy'.format(pp_path, cnum)) else 'h5'
    pp_path = '{}.{}'.format(pp_path, file_ext)
    if not os.path.exists(pp_path):
        raise ValueError('Preprocess file not found for case {}'.format(cnum))
    return pp_path

def get_start_idx(vol):
    start_idx = 0
    inc_idx = 1
    if vol.shape[0] <= 225:
        start_idx = int(vol.shape[0] * 0.12)
        inc_idx = 3
    else:
        start_idx = int(vol.shape[0] * 0.15)
        inc_idx = 5
    return start_idx, inc_idx

def process_vol(vol):
    vol = vol / vol.mean()
    vol = np.clip(vol, 0, vol.max())
    vol = np.interp(vol, (vol.min(), vol.max()), (0, 1))
    
    st, inc_idx = get_start_idx(vol)
    vol = vol[st:-st]
    vol = vol[::inc_idx]
    return vol

/home/srivathsa/projects/SubtleGad/train/subtle/utils/hyperparameter.py:8: UserWarning: Module test_tube not found - hyperparameter related functions cannot be used
  warnings.warn('Module test_tube not found - hyperparameter related functions cannot be used')


<Figure size 432x288 with 0 Axes>

### View Inference Cases

In [13]:
src_path = '/home/srivathsa/projects/studies/gad/inf_qview'
cases = sorted([f.split('/')[-1].replace('.npy', '') 
                for f in glob('{}/*.npy'.format(src_path))])

read_cases = [c for c in open('/home/srivathsa/projects/studies/gad/inf_read_cases.txt', 'r').read().split('\n')
              if len(c) > 0]
# read_cases = []
cases = ['Select case number...'] + [c for c in cases if c not in read_cases]

print('{} cases remaining'.format(len(cases)))

drop_down = widgets.Dropdown(
    options=cases,
    value=cases[0],
    description='Case:',
)

read_btn = widgets.Button(description="Mark as read")

def mark_read(arg):
    with open('/home/srivathsa/projects/studies/gad/inf_read_cases.txt', 'a') as status_f:
        status_f.write(drop_down.value + '\n')

def show_gad_gt(idx):
    plt.clf()
    img_disp = np.hstack([cur_vol[0, idx], cur_vol[1, idx]])
    plt.imshow(img_disp)
    plt.title('Slice {}'.format(idx))
    plt.axis('off')

def on_dd_change(chg):
    global cur_vol
    if chg['type'] == 'change' and chg['name'] == 'value':
        if 'Select' in chg['new']: return
        clear_output()
        cur_vol = np.load('{}/{}.npy'.format(src_path, str(chg['new'])))
        display(drop_down)
        display(read_btn)
        interact(show_gad_gt, idx=widgets.IntSlider(min=0, max=cur_vol.shape[1]-1, step=1, value=0))

drop_down.observe(on_dd_change)
display(drop_down)
display(read_btn)
read_btn.on_click(mark_read)
global cur_vol
cur_vol = None

Dropdown(description='Case:', index=1, options=('Select case number...', 'NO57', 'NO65', 'Patient_0241', 'Pati…

Button(description='Mark as read', style=ButtonStyle())

interactive(children=(IntSlider(value=0, description='idx', max=49), Output()), _dom_classes=('widget-interact…

In [11]:
cases = ['NO57', 'NO65']

df_inf = pd.read_csv('/home/srivathsa/projects/studies/gad/gad_inf_repo.csv')
# df_inf = df_inf[df_inf['Read?'] == 'No']
df_inf = df_inf.query('Case in @cases')
df_inf = df_inf.rename(columns={"Inference Path (dgx-1)": "Inference_Path"})

dest_path = '/home/srivathsa/projects/studies/gad/inf_qview'

inf_list = df_inf.to_dict(orient='records')
proc_cases = sorted([f.split('/')[-1].replace('.npy', '') for f in glob('{}/*.npy'.format(dest_path))])
inf_list = [rec for rec in inf_list if rec['Case'] not in proc_cases]

In [12]:
for inf_rec in tqdm(inf_list, total=len(inf_list)):
    cnum = inf_rec['Case']
    try:
        if 'LOCAL' in inf_rec['Inference_Path']:
            inf_rec['Inference_Path'] = inf_rec['Inference_Path'].replace(
                'LOCAL:/', '/home/srivathsa/projects/studies/gad/stanford/data/'
            )
        inf_path = '{}/{}/{}_SubtleGad'.format(inf_rec['Inference_Path'], cnum, cnum)
        
        pp_path = get_pp_path(inf_rec)

        gt_raw = suio.load_file(pp_path)[:, 2]
        gad_raw = get_dicom_vol(inf_path)

        if gt_raw.shape[0] != gad_raw.shape[0]:
            gt_raw = gt_raw[::2]

        gt = process_vol(gt_raw)
        gad = process_vol(gad_raw)

        if not np.array_equal(gt.shape, gad.shape):
            gad = supre.zero_pad(gad, target_size=256)
        new_arr = np.array([gt, gad])
        np.save('{}/{}.npy'.format(dest_path, cnum), new_arr)
    except Exception as e:
        print('ERROR in {}: {}'.format(cnum, e))

/home/srivathsa/miniconda3/envs/py35gad/lib/python3.6/site-packages/ipykernel_launcher.py:1: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  """Entry point for launching an IPython kernel.


  0%|          | 0/2 [00:00<?, ?it/s]

In [ ]:
def show_gad_gt(idx):
    plt.imshow(np.hstack([gt[idx], gad[idx]]))
    plt.title('Slice {}'.format(idx))
    plt.axis('off')
interact(show_gad_gt, idx=widgets.IntSlider(min=0, max=gt.shape[0]-1, step=1, value=0))

In [ ]:
bpath = '/home/srivathsa/projects/studies/gad/tiantan/data'
cases = sorted([d.split('/')[-1] for d in glob('/home/srivathsa/projects/studies/gad/tiantan/data/NO*') 
                if 'mod' not in d and 'mc' not in d])

ids = []

for case in cases:
    fpaths = [f for f in glob('{}/{}/**/*.dcm'.format(bpath, case), recursive=True)]
    if len(fpaths) == 0:
        fpaths = [f for f in glob('{}/{}/**/Mag*'.format(bpath, case), recursive=True)]
    fpath_dcm = fpaths[0]
    dcm = pydicom.dcmread(fpath_dcm)
    ids.append((case, dcm.StudyInstanceUID))

In [ ]:
for cnum, study_uid in ids:
    dup_row = [c for c, s in ids if c != cnum and s == study_uid]
    if len(dup_row) > 0:
        print(cnum, dup_row[0])

### AUC/ROC Curve

In [ ]:
from sklearn.metrics import accuracy_score, f1_score, roc_curve, auc

y_test = np.append(np.ones(295), np.zeros(296))
y_score = np.zeros(22)
y_score = np.append(y_score, np.ones(273))
y_score = np.append(y_score, np.ones(24))
y_score = np.append(y_score, np.zeros(272))

In [ ]:
fpr, tpr, nt = roc_curve(y_test, y_score)

In [ ]:
plt.plot(fpr, tpr)